In [76]:
:dep ndarray
:dep once_cell
:dep ndarray-rand
:dep rand_isaac

use ndarray::prelude::*;
use ndarray::{concatenate, Axis};
use once_cell::sync::Lazy;
use ndarray_rand::RandomExt;
use ndarray_rand::rand_distr::Uniform;
use rand_isaac::isaac64::Isaac64Rng;
use ndarray_rand::rand::SeedableRng;

In [70]:
const L_SIZE: usize = 100;

static UP_ROLL_SLC: Lazy<[usize; L_SIZE]> = Lazy::new(|| {  // left
    let mut arr = [0_usize; L_SIZE];
    for i in 0..L_SIZE - 1 {
        arr[i] = i + 1;
    }
    arr[L_SIZE - 1] = 0;
    arr
});
static DOWN_ROLL_SLC: Lazy<[usize; L_SIZE]> = Lazy::new(|| {  // right
    let mut arr = [0_usize; L_SIZE];
    for i in 1..L_SIZE {
        arr[i] = i - 1;
    }
    arr[0] = L_SIZE - 1;
    arr
});

enum Direction {
    Up,
    Down,
    Left,
    Right,
}

fn roll(arr: &Array<f64, Ix2>, direction: Direction) -> Array<f64, Ix2> {
    match direction{
        Direction::Up => {
            concatenate![Axis(0), arr.slice(s![1.., ..]), arr.slice(s![0..1, ..])]
        },
        Direction::Down => {
            concatenate![Axis(0), arr.slice(s![L_SIZE-1.., ..]), arr.slice(s![..L_SIZE-1, ..])]
        },
        Direction::Left => {
            concatenate![Axis(1), arr.slice(s![.., 1..]), arr.slice(s![.., 0..1])]
        },
        Direction::Right => {
            concatenate![Axis(1), arr.slice(s![.., L_SIZE-1..]), arr.slice(s![.., ..L_SIZE-1])]
        },
    }
}

fn nabla2C(c: &Array<f64, Ix2>) -> Array<f64, Ix2> {
    (
        roll(c, Direction::Up) + 
        roll(c, Direction::Down) + 
        roll(c, Direction::Left) + 
        roll(c, Direction::Right) - 
        4. * c
    ) / 4.
}

fn cartesian_product(arr1: &Array<f64, ndarray::Ix1>, arr2: &Array<f64, ndarray::Ix1>) -> Array2<f64> {
    let (n, m) = (arr1.len(), arr2.len());
    let mut product = Array2::<f64>::zeros((n * m, 2));

    for (i, val1) in arr1.iter().enumerate() {
        for (j, val2) in arr2.iter().enumerate() {
            product[[i * m + j, 0]] = *val1;
            product[[i * m + j, 1]] = *val2;
        }
    }

    product
}


#[derive(Default)]
pub struct PatternFormationConfig {
    positionX: Array<f64, Ix2>,
    phaseTheta: Array<f64, Ix1>,
    cemoU: Array<f64, Ix2>,
    cemoV: Array<f64, Ix2>,
    boundaryLength: f64,
    cellNumInLine: usize,
    cemoPositon: Array<f64, Ix3>,
    dx: f64,
    agentsNum: usize,
    productRateKU: f64,
    productRateKV: f64,
    decayRateKd: f64,
    feedRateKf: f64,
    diffusionRateDU: f64,
    diffusionRateDV: f64,
    chemoBetaU: f64,
    chemoBetaV: f64,
    u0: f64,
    v0: f64,
    dt: f64,
    speedV: f64,
    couplingK: f64,
    alpha: f64,
}

impl PatternFormation {
    fn new(couplingK: f64, alpha: f64, u0: f64, v0: f64, dt: f64, boundaryLength: f64,
           decayRateKd: f64, feedRateKf: f64, productRateKU: f64, productRateKV: f64,
           diffusionRateDU: f64, diffusionRateDV: f64, chemoBetaU: f64, chemoBetaV: f64, 
           agentsNum: usize, cellNumInLine: usize, speedV: f64, randomSeed: u64) -> Self {
        let mut rng = Isaac64Rng::seed_from_u64(randomSeed);
        let positionX = Array::random_using((agentsNum, 2), Uniform::new(0., boundaryLength), &mut rng);
        let phaseTheta = Array::random_using(agentsNum, Uniform::new(0., 2. * std::f64::consts::PI), &mut rng);
        let cemoU = Array::ones((cellNumInLine, cellNumInLine)) * u0;
        let cemoV = Array::ones((cellNumInLine, cellNumInLine)) * v0;
        let cemoPositon = cartesian_product(
            &Array::linspace(0., boundaryLength, cellNumInLine),
            &Array::linspace(0., boundaryLength, cellNumInLine)
        ).into_shape((cellNumInLine, cellNumInLine, 2)).unwrap();
        let dx = boundaryLength / cellNumInLine as f64;

        PatternFormation {
            positionX, phaseTheta, cemoU, cemoV,
            boundaryLength,
            cellNumInLine,
            cemoPositon,
            dx,
            agentsNum,
            productRateKU,
            productRateKV,
            decayRateKd,
            feedRateKf,
            diffusionRateDU,
            diffusionRateDV,
            chemoBetaU,
            chemoBetaV,
            u0,
            v0,
            dt,
            speedV,
            couplingK,
            alpha,
        }
    }
}

impl Default for PatternFormation {
    fn default() -> Self {
        PatternFormation{
            positionX: Array::zeros((0, 0)),
            phaseTheta: Array::zeros(0),
            cemoU: Array::zeros((0, 0)),
            cemoV: Array::zeros((0, 0)),
            boundaryLength: 0.,
            cellNumInLine: 0,
            cemoPositon: Array::zeros((0, 0, 0)),
            dx: 0.,
            agentsNum: 0,
            productRateKU: 0.,
            productRateKV: 0.,
            decayRateKd: 0.,
            feedRateKf: 0.,
            diffusionRateDU: 0.,
            diffusionRateDV: 0.,
            chemoBetaU: 0.,
            chemoBetaV: 0.,
            u0: 0.,
            v0: 0.,
            dt: 0.,
            speedV: 0.,
            couplingK: 0.,
            alpha: 0.,
        }
    }
}

let c: Array<f64, Ix2> = Array::zeros((L_SIZE, L_SIZE));
// let c: Array<f64, Ix2> = Array::from_shape_vec((3, 3), vec![1., 2., 3., 4., 5., 6., 7., 8., 9.]).unwrap();

In [100]:
const L_SIZE: usize = 100;

static UP_ROLL_SLC: Lazy<[usize; L_SIZE]> = Lazy::new(|| {  // left
    let mut arr = [0_usize; L_SIZE];
    for i in 0..L_SIZE - 1 {
        arr[i] = i + 1;
    }
    arr[L_SIZE - 1] = 0;
    arr
});
static DOWN_ROLL_SLC: Lazy<[usize; L_SIZE]> = Lazy::new(|| {  // right
    let mut arr = [0_usize; L_SIZE];
    for i in 1..L_SIZE {
        arr[i] = i - 1;
    }
    arr[0] = L_SIZE - 1;
    arr
});

enum Direction {
    Up,
    Down,
    Left,
    Right,
}

fn roll(arr: &Array<f64, Ix2>, direction: Direction) -> Array<f64, Ix2> {
    match direction{
        Direction::Up => {
            concatenate![Axis(0), arr.slice(s![1.., ..]), arr.slice(s![0..1, ..])]
        },
        Direction::Down => {
            concatenate![Axis(0), arr.slice(s![L_SIZE-1.., ..]), arr.slice(s![..L_SIZE-1, ..])]
        },
        Direction::Left => {
            concatenate![Axis(1), arr.slice(s![.., 1..]), arr.slice(s![.., 0..1])]
        },
        Direction::Right => {
            concatenate![Axis(1), arr.slice(s![.., L_SIZE-1..]), arr.slice(s![.., ..L_SIZE-1])]
        },
    }
}

fn nabla2C(c: &Array<f64, Ix2>) -> Array<f64, Ix2> {
    (
        roll(c, Direction::Up) + 
        roll(c, Direction::Down) + 
        roll(c, Direction::Left) + 
        roll(c, Direction::Right) - 
        4. * c
    ) / 4.
}

fn cartesian_product(arr1: &Array<f64, ndarray::Ix1>, arr2: &Array<f64, ndarray::Ix1>) -> Array2<f64> {
    let (n, m) = (arr1.len(), arr2.len());
    let mut product = Array2::<f64>::zeros((n * m, 2));

    for (i, val1) in arr1.iter().enumerate() {
        for (j, val2) in arr2.iter().enumerate() {
            product[[i * m + j, 0]] = *val1;
            product[[i * m + j, 1]] = *val2;
        }
    }

    product
}


pub struct PatternFormationConfig {
    couplingK: f64, alpha: f64, u0: f64, v0: f64, dt: f64, boundaryLength: f64,
    decayRateKd: f64, feedRateKf: f64, productRateKU: f64, productRateKV: f64,
    diffusionRateDU: f64, diffusionRateDV: f64, chemoBetaU: f64, chemoBetaV: f64, 
    agentsNum: usize, cellNumInLine: usize, speedV: f64, randomSeed: u64
}

impl Default for PatternFormationConfig {
    fn default() -> Self {
        PatternFormationConfig{
            couplingK: 0.,
            alpha: 0.,
            u0: 0.012,
            v0: 0.012,
            dt: 0.02,
            boundaryLength: 10.,
            decayRateKd: 1.,
            feedRateKf: 1.,
            productRateKU: 1.,
            productRateKV: 1.,
            diffusionRateDU: 0.01,
            diffusionRateDV: 0.01,
            chemoBetaU: 1.,
            chemoBetaV: 1.,
            agentsNum: 1000,
            cellNumInLine: 250,
            speedV: 3.,
            randomSeed: 10,
        }
    }
}
impl std::fmt::Debug for PatternFormationConfig {
    fn fmt(&self, f: &mut std::fmt::Formatter<'_>) -> std::fmt::Result {
        f.debug_struct("PatternFormationConfig")
            .field("couplingK", &self.couplingK)
            .field("alpha", &self.alpha)
            .field("u0", &self.u0)
            .field("v0", &self.v0)
            .field("dt", &self.dt)
            .field("boundaryLength", &self.boundaryLength)
            .field("decayRateKd", &self.decayRateKd)
            .field("feedRateKf", &self.feedRateKf)
            .field("productRateKU", &self.productRateKU)
            .field("productRateKV", &self.productRateKV)
            .field("diffusionRateDU", &self.diffusionRateDU)
            .field("diffusionRateDV", &self.diffusionRateDV)
            .field("chemoBetaU", &self.chemoBetaU)
            .field("chemoBetaV", &self.chemoBetaV)
            .field("agentsNum", &self.agentsNum)
            .field("cellNumInLine", &self.cellNumInLine)
            .field("speedV", &self.speedV)
            .field("randomSeed", &self.randomSeed)
            .finish()
    }
}

pub struct PatternFormation {
    positionX: Array<f64, Ix2>, phaseTheta: Array<f64, Ix1>,
    cemoU: Array<f64, Ix2>, cemoV: Array<f64, Ix2>,
    boundaryLength: f64,
    halfBoundaryLength: f64,
    cellNumInLine: usize,
    cemoPositon: Array<f64, Ix3>,
    dx: f64,
    agentsNum: usize,
    productRateKU: f64,
    productRateKV: f64,
    decayRateKd: f64,
    feedRateKf: f64,
    diffusionRateDU: f64,
    diffusionRateDV: f64,
    chemoBetaU: f64,
    chemoBetaV: f64,
    u0: f64,
    v0: f64,
    dt: f64,
    speedV: f64,
    couplingK: f64,
    alpha: f64,
    config: PatternFormationConfig,
}


impl PatternFormation {
    fn new(config: PatternFormationConfig) -> Self {
        let mut rng = Isaac64Rng::seed_from_u64(config.randomSeed);
        let positionX = Array::random_using(
            (config.agentsNum, 2), 
            Uniform::new(0., config.boundaryLength), 
            &mut rng
        );
        let phaseTheta = Array::random_using(
            config.agentsNum, Uniform::new(0., 2. * std::f64::consts::PI), &mut rng);
        let cemoU = Array::ones((config.cellNumInLine, config.cellNumInLine)) * config.u0;
        let cemoV = Array::ones((config.cellNumInLine, config.cellNumInLine)) * config.v0;
        let cemoPositon = cartesian_product(
            &Array::linspace(0., config.boundaryLength, config.cellNumInLine),
            &Array::linspace(0., config.boundaryLength, config.cellNumInLine)
        ).into_shape((config.cellNumInLine, config.cellNumInLine, 2)).unwrap();
        let dx = config.boundaryLength / config.cellNumInLine as f64;
        let halfBoundaryLength = config.boundaryLength / 2.;

        PatternFormation {
            positionX: positionX, 
            phaseTheta: phaseTheta, 
            cemoU: cemoU, 
            cemoV: cemoV,
            boundaryLength: config.boundaryLength,
            halfBoundaryLength: halfBoundaryLength,
            cellNumInLine: config.cellNumInLine,
            cemoPositon: cemoPositon,
            dx: dx,
            agentsNum: config.agentsNum,
            productRateKU: config.productRateKU,
            productRateKV: config.productRateKV,
            decayRateKd: config.decayRateKd,
            feedRateKf: config.feedRateKf,
            diffusionRateDU: config.diffusionRateDU,
            diffusionRateDV: config.diffusionRateDV,
            chemoBetaU: config.chemoBetaU,
            chemoBetaV: config.chemoBetaV,
            u0: config.u0,
            v0: config.v0,
            dt: config.dt,
            speedV: config.speedV,
            couplingK: config.couplingK,
            alpha: config.alpha,
            config: config,
        }
    }
}
impl std::fmt::Debug for PatternFormation {
    fn fmt(&self, f: &mut std::fmt::Formatter<'_>) -> std::fmt::Result {
        f.debug_struct("PatternFormation on")
            .field("", &self.config)
            .finish()
    }
}


let c: Array<f64, Ix2> = Array::zeros((L_SIZE, L_SIZE));
// let c: Array<f64, Ix2> = Array::from_shape_vec((3, 3), vec![1., 2., 3., 4., 5., 6., 7., 8., 9.]).unwrap();

In [168]:
// let config = PatternFormationConfig::default();
let mut model = PatternFormation::new(PatternFormationConfig::default());
let others = model.positionX.clone().insert_axis(Axis(1));
let subX = model.positionX.clone() - others;
model

PatternFormation on { : PatternFormationConfig { couplingK: 0.0, alpha: 0.0, u0: 0.012, v0: 0.012, dt: 0.02, boundaryLength: 10.0, decayRateKd: 1.0, feedRateKf: 1.0, productRateKU: 1.0, productRateKV: 1.0, diffusionRateDU: 0.01, diffusionRateDV: 0.01, chemoBetaU: 1.0, chemoBetaV: 1.0, agentsNum: 1000, cellNumInLine: 250, speedV: 3.0, randomSeed: 10 } }

In [158]:
let c: Array<f64, Ix2> = Array::zeros((L_SIZE, 2));
let c_ = c.clone().insert_axis(Axis(1));
// c - c.insert_axis(Axis(1))
c

[[0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],
 [0.0, 0.0],

In [9]:
let a = Array::linspace(0., 10., 11);
let b = Array::linspace(0., 10., 11);

In [68]:
let seed: u64 = 42;
let mut rng = Isaac64Rng::seed_from_u64(seed);
//  = Isaac64Rng::seed_from_u64(seed);

// Generate a random array using `rng`
let a = Array::random_using((2, 5), Uniform::new(0., 10.), &mut rng);
Array::random_using((2, 5), Uniform::new(0., 10.), &mut rng)

[[3.1181959423520778, 7.398254113552798, 0.9054507828914282, 7.288312058240056, 0.46979328616078453],
 [5.84661463696243, 6.0534170114191195, 2.310179777619814, 6.67102895045365, 9.282527701718191]], shape=[2, 5], strides=[5, 1], layout=Cc (0x5), const ndim=2

In [55]:
Array::random_using((2, 5), Uniform::new(0., 10.), &mut rng)

[[4.709879346094081, 7.781792332881869, 7.280398704422922, 2.4949100191426288, 4.117271206212072],
 [7.624596970211634, 7.920856030429395, 6.8138502792472995, 6.713713388957947, 8.515178964314147]], shape=[2, 5], strides=[5, 1], layout=Cc (0x5), const ndim=2

In [58]:
rng

Isaac64Rng(BlockRng64 { core: Isaac64Core {}, result_len: 256, index: 20, half_used: false })

In [57]:
Array::random_using((2, 5), Uniform::new(0., 10.), &mut rng)

[[3.1181959423520778, 7.398254113552798, 0.9054507828914282, 7.288312058240056, 0.46979328616078453],
 [5.84661463696243, 6.0534170114191195, 2.310179777619814, 6.67102895045365, 9.282527701718191]], shape=[2, 5], strides=[5, 1], layout=Cc (0x5), const ndim=2

In [37]:
Isaac64Rng // ::seed_from_u64(seed);

Error: expected `;`, found <end of input>

Error: use of deprecated associated function `ndarray::impl_methods::<impl ndarray::ArrayBase<S, D>>::into_shape`: Use `.into_shape_with_order()` or `.to_shape()`

In [163]:
concatenate![Axis(0), c.slice(s![1.., ..]), c.slice(s![0..1, ..])]

[[4.0, 5.0, 6.0],
 [7.0, 8.0, 9.0],
 [1.0, 2.0, 3.0]], shape=[3, 3], strides=[3, 1], layout=Cc (0x5), const ndim=2

In [185]:
nabla2C(&c)

[[3.0, 2.25, 1.5],
 [0.75, 0.0, -0.75],
 [-1.5, -2.25, -3.0]], shape=[3, 3], strides=[3, 1], layout=Cc (0x5), const ndim=2

In [4]:
use std::time::{Instant};
let counts: usize = 10000;
fn func(c: &Array<f64, Ix2>) {
    // c.select(Axis(0), &*UP_ROLL_SLC);
    nabla2C(&c);
    // concatenate![Axis(0), c.slice(s![1.., ..]), c.slice(s![0..1, ..])];
}

let start = Instant::now();
func(&c);
let mut duration = start.elapsed();
for _ in 0..(counts - 1) {
    let start = Instant::now();
    func(&c);
    duration += start.elapsed();
}
println!("mean duration: {:?}", duration / counts as u32);

mean duration: 76.285µs


In [130]:
c.select(Axis(1), &*DOWN_ROLL_SLC2)

[[2.0, 3.0, 1.0],
 [5.0, 6.0, 4.0],
 [8.0, 9.0, 7.0]], shape=[3, 3], strides=[1, 3], layout=Ff (0xa), const ndim=2

In [22]:
testC

[[1.0, 2.0],
 [3.0, 4.0]], shape=[2, 2], strides=[2, 1], layout=Cc (0x5), const ndim=2

In [131]:
// 实现类似numpy.roll的功能


// roll(&testC, 1)

In [137]:
roll(&c, Direction::Right)

[[3.0, 1.0, 2.0],
 [6.0, 4.0, 5.0],
 [9.0, 7.0, 8.0]], shape=[3, 3], strides=[1, 3], layout=Ff (0xa), const ndim=2